In [8]:
import requests
from bs4 import BeautifulSoup
import time
response = requests.get("https://en.wikipedia.org/wiki/Heroldsberg")
soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the main content of the article
content_div = soup.find(id='mw-content-text')
    
# Ignore IPA, pronunciations, and citation links
for span in content_div.find_all(['span', 'sup']):
        span.decompose()
    
    # Find the introduction content, which is usually the first few paragraphs
intro_paragraphs = []
for elem in content_div.find_all(['p', 'ul', 'ol']):
        if elem.find_previous_sibling(['h2', 'h3', 'h4', 'h5', 'h6']):
            break
        intro_paragraphs.append(elem)
    
    # Ignore links within parentheses in the introduction
for paragraph in intro_paragraphs:
        for text in paragraph.find_all(string=True):
            if '(' in text and ')' in text:
                text.replace_with(text.split('(')[0] + text.split(')')[-1])
    
    # Get the first valid link in the introduction
for paragraph in intro_paragraphs:
        for link in paragraph.find_all('a', href=True):
            href = link['href']
            if (href.startswith('/wiki/') and 
                not href.startswith('/wiki/Help:') and 
                not href.startswith('/wiki/Wikipedia:') and 
                not href.startswith('/wiki/Template:')):
                print('https://en.wikipedia.org' + href)

https://en.wikipedia.org/wiki/East_Franconian_German
https://en.wikipedia.org/wiki/Erlangen-H%C3%B6chstadt
https://en.wikipedia.org/wiki/Bavaria
https://en.wikipedia.org/wiki/Germany
https://en.wikipedia.org/wiki/Nuremberg
https://en.wikipedia.org/wiki/Erlangen
https://en.wikipedia.org/wiki/Schwan-Stabilo


In [6]:
content_div

<div class="mw-body-content" id="mw-content-text"><div class="mw-content-ltr mw-parser-output" dir="ltr" lang="en"><div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Municipality in Bavaria, Germany</div><style data-mw-deduplicate="TemplateStyles:r1229112069">.mw-parser-output .infobox-subbox{padding:0;border:none;margin:-3px;width:auto;min-width:100%;font-size:100%;clear:none;float:none;background-color:transparent}.mw-parser-output .infobox-3cols-child{margin:auto}.mw-parser-output .infobox .navbar{font-size:100%}body.skin-minerva .mw-parser-output .infobox-header,body.skin-minerva .mw-parser-output .infobox-subheader,body.skin-minerva .mw-parser-output .infobox-above,body.skin-minerva .mw-parser-output .infobox-title,body.skin-minerva .mw-parser-output .infobox-image,body.skin-minerva .mw-parser-output .infobox-full-data,body.skin-minerva .mw-parser-output .infobox-below{text-align:center}html.skin-theme-clientpref-night .mw-parser-output .infobo

In [15]:
import requests
from bs4 import BeautifulSoup
import time

def get_first_link(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the main content of the article
    content_div = soup.find(id='mw-content-text')
    
    # Ignore IPA, pronunciations, and citation links
    for span in content_div.find_all(['span', 'sup']):
        span.decompose()
    
    # Find the introduction content, which is usually the first few paragraphs
    intro_paragraphs = []
    for elem in content_div.find_all(['p', 'ul', 'ol']):
        if elem.find_previous_sibling(['h2', 'h3', 'h4', 'h5', 'h6']):
            break
        intro_paragraphs.append(elem)
    
    # Remove text within parentheses
    for paragraph in intro_paragraphs:
        for text in paragraph.find_all(string=True):
            if '(' in text and ')' in text:
                text.replace_with(text.split('(')[0] + text.split(')')[-1])
    
    # Get the first valid link in the introduction
    for paragraph in intro_paragraphs:
        for link in paragraph.find_all('a', href=True):
            href = link['href']
            if (href.startswith('/wiki/') and 
                not href.startswith('/wiki/Help:') and 
                not href.startswith('/wiki/Wikipedia:') and 
                not href.startswith('/wiki/Template:')):
                return 'https://en.wikipedia.org' + href
    return None

def follow_links(start_url, max_jumps=100):
    visited = set()
    current_url = start_url
    steps = 0
    
    while current_url and steps < max_jumps:
        if current_url == "https://en.wikipedia.org/wiki/Philosophy":
            print("Philosopy reached")
            break
        if current_url in visited:
            print(f"Loop detected at {current_url}.")
            break
        visited.add(current_url)
        print(f"Step {steps + 1}: {current_url}")
        
        next_url = get_first_link(current_url)
        if not next_url:
            print(f"Dead end reached at {current_url}.")
            break
        
        current_url = next_url
        steps += 1
        time.sleep(1)  # To prevent overwhelming the server
    
    if steps >= max_jumps:
        print("Maximum number of jumps reached.")

# Start with a random Wikipedia page
start_url = 'https://en.wikipedia.org/wiki/cricket'
follow_links(start_url)


Step 1: https://en.wikipedia.org/wiki/cricket
Step 2: https://en.wikipedia.org/wiki/Test_cricket
Step 3: https://en.wikipedia.org/wiki/Women%27s_Test_cricket
Step 4: https://en.wikipedia.org/wiki/Women%27s_cricket
Loop detected at https://en.wikipedia.org/wiki/Test_cricket.
